# Converting the csv files to a SQL database

In [1]:
import pandas as pd
import sqlite3
import os
import re

In [2]:
%load_ext autoreload
%autoreload 2

# my code
from SQLutils import addCSVToSQLdb, createNewSQLdbFromCSVs

## Create new database files with the desired paragraph(s)

In [ ]:
allPara = pd.read_csv('../csv/paragraphs.csv')
allParaName = allPara['paragraphname'].to_list()
allParaName

In [ ]:
#all that we currently have
createNewSQLdbFromCSVs('Dec2021_all', allParaName)

In [ ]:
#default, available on load
default = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54']
createNewSQLdbFromCSVs('default', default)

In [ ]:
createNewSQLdbFromCSVs('testing', default)

In [ ]:
useCases = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54',
    'CJE BCPs for DSA',
    'Co_alloy_WhitneyT',
    'Cobalt-Based Superalloys_Brandonfinished',
    'Metal AM - SLM',
    'PECs',
    'PECs_class',
    'Superalloy -test',
    'Thermoelectrics_JM',
    'whitney DSA',
    'CJE DSA Random BCPs'
]
createNewSQLdbFromCSVs('CHiMaD_UseCases', useCases)

# Create the metrics database

In [ ]:
dbName = 'CHiMaD_metrics.db'
tableName = 'loginMetrics'
columns = ['timestamp', 'name', 'organization', 'email', 'purpose', 'groupname']

In [ ]:
# connect to the SQL database (this will create a file if it does not already exist)
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

# include all columns, and assume that they are all text
# allow only alphanumeric values in column names
cols = ''
for col in columns:
    cc = re.sub(r'\W+', '', col) 
    cols += cc + ' text, ' 
cols = cols[:-2]
    
# create the table
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

cursor.close()

# Remove a table from a file

In [ ]:
dbName = 'default.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

dropTableStatement = "DROP TABLE dbs"
cursor.execute(dropTableStatement)
cursor.close()

# Remove a row from a database file

In [ ]:
#read in table 
dbName = 'default.db'
tablename = 'polymercompositeexample'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

# perform selection
df = df.loc[~((df['username'] == 'AaronGeller') & (df.index > 41))]
# rewrite table
df.to_sql(tablename, conn, if_exists='replace', index = False)
display(df)

cursor.close()

# Copy a paragraph from one database to another

In [31]:
#copy the paragraph 
dbName1 = 'testing.db'
dbName2 = 'mse390.db'
paragraphs = 'paragraphs'
tablename = 'group5'

# get the row from the first db
conn1 = sqlite3.connect(dbName1)
cursor1 = conn1.cursor()
cursor1.execute('SELECT * FROM ' + paragraphs)
columns1 = [description[0] for description in cursor1.description]
df1 = pd.DataFrame(cursor1.fetchall(), columns = columns1)  
display(df1)

row = df1.loc[df1['paragraphname'] == tablename]
display(row)

# add that to the second db
conn2 = sqlite3.connect(dbName2)
cursor2 = conn2.cursor()
cursor2.execute('SELECT * FROM ' + paragraphs)
columns2 = [description[0] for description in cursor2.description]
df2 = pd.DataFrame(cursor2.fetchall(), columns = columns2)  
dfJoined = pd.concat([df2, row])
display(df2)
display(dfJoined)

# rewrite the file
dfJoined.to_sql(paragraphs, conn2, if_exists='replace', index = False)


#then copy the table
conn1 = sqlite3.connect(dbName1)
cursor1 = conn1.cursor()
cursor1.execute('SELECT * FROM ' + tablename)
columns1 = [description[0] for description in cursor1.description]
df1 = pd.DataFrame(cursor1.fetchall(), columns = columns1)  
display(df1)
df1.to_sql(tablename, conn2, if_exists='replace', index = False)

,paragraphname,paragraph,answersJSON
0,Polymer Composite Example,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample"",""t..."
1,Aluminum-CHiMaDExample,An aluminum alloy is prepared by [[ingot metal...,"[{""paragraphname"":""aluminumchimadexample"",""tas..."
2,Thermoelectrics_test,The efficiency of thermoelectric devices is re...,"[{""paragraphname"":""thermoelectricstest"",""task""..."
3,Emery-Exam1-M54,"In 2007, the US Navy contracted QuestTek to de...","[{""paragraphname"":""emeryexam1m54"",""task"":""para..."
4,Al_test,An aluminum alloy is prepared by ingot metallu...,"[{""paragraphname"":""altest"",""task"":""para"",""ingo..."
5,ClayExample1,An aluminum alloy is prepared by ingot metallu...,"[{""paragraphname"":""clayexample1"",""task"":""para""..."
6,testing1,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""testing1"",""task"":""para"",""li..."
7,kstest,Polymer matrix composites are traditionally va...,
8,group2,The efficiency of thermoelectric devices is re...,
9,group3,Polymer matrix composites are traditionally va...,


,paragraphname,paragraph,answersJSON
10,group5,The efficiency of thermoelectric devices is re...,


,paragraphname,paragraph,answersJSON
0,Polymer Composite Example,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample"",""t..."
1,Aluminum-CHiMaDExample,An aluminum alloy is prepared by [[ingot metal...,"[{""paragraphname"":""aluminumchimadexample"",""tas..."
2,Thermoelectrics_test,The efficiency of thermoelectric devices is re...,"[{""paragraphname"":""thermoelectricstest"",""task""..."
3,Emery-Exam1-M54,"In 2007, the US Navy contracted QuestTek to de...","[{""paragraphname"":""emeryexam1m54"",""task"":""para..."
4,mse390group4,[[Layered two-dimensional]] nanomaterials inte...,"[{""paragraphname"":""mse390group4"",""task"":""para""..."
5,group1,Concrete is traditionally valued for use in [[...,
6,group2,The efficiency of thermoelectric devices is re...,
7,group3,Polymer matrix composites are traditionally va...,


,paragraphname,paragraph,answersJSON
0,Polymer Composite Example,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample"",""t..."
1,Aluminum-CHiMaDExample,An aluminum alloy is prepared by [[ingot metal...,"[{""paragraphname"":""aluminumchimadexample"",""tas..."
2,Thermoelectrics_test,The efficiency of thermoelectric devices is re...,"[{""paragraphname"":""thermoelectricstest"",""task""..."
3,Emery-Exam1-M54,"In 2007, the US Navy contracted QuestTek to de...","[{""paragraphname"":""emeryexam1m54"",""task"":""para..."
4,mse390group4,[[Layered two-dimensional]] nanomaterials inte...,"[{""paragraphname"":""mse390group4"",""task"":""para""..."
5,group1,Concrete is traditionally valued for use in [[...,
6,group2,The efficiency of thermoelectric devices is re...,
7,group3,Polymer matrix composites are traditionally va...,
10,group5,The efficiency of thermoelectric devices is re...,


,Timestamp,IP,username,version,task,figureofmeritzt,latticethermalconductivity,electronicthermalconductivity,carriermobility,carrierconcentration,...,densityofstates,dopingthematrix,latticethermalconductivityresistance,microstructuregrainboundariessecondaryphases,castthealloyasaningot,consolidationhomogenizationstepiesparkplasmasinteringhotpressing,annealingtreatment,grainstructureincludinggbchemistry,nanostructureofthesecondaryphases,phononscattering


## Test selecting everything from the database and puting back into DataFrame

In [ ]:
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]

for t in tables:
    print(t)
    cursor.execute('SELECT * FROM ' + t)
    columns = [description[0] for description in cursor.description]
    df2 = pd.DataFrame(cursor.fetchall(), columns = columns)    
    display(df2)


In [ ]:
cursor.close()

# Manual testing

## Read in a csv file

In [ ]:
#csvName = 'thermoelectricsjm.csv' # this has a column with ".1" in the name, and sqlite does not allow it
csvName = 'paragraphs.csv' # this has a column with ".1" in the name, and sqlite does not allow it
df = pd.read_csv('../csv/' + csvName)
df

## Create a SQL database

In [ ]:
dbName = 'CHiMaD_SDC.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

In [ ]:
# the database name will be the file name w/o csv
# take only alphanumeric values
tableName = re.sub(r'\W+', '',csvName.replace('.csv','')) 
tableName

In [ ]:
# include all columns, and assume that they are all text
cols = ''
for col in df.columns:
    cc = re.sub(r'\W+', '', col) # only alphanumeric values in column names
    cols += cc + ' text, '
cols = cols[:-2]
cols

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

In [ ]:
# add the dataframe into the database
df.to_sql(tableName, conn, if_exists='replace', index = False)

In [ ]:
#grab one table and write to file
t = 'paragraphs'
cursor.execute('SELECT * FROM ' + t)
columns = [description[0] for description in cursor.description]
df2 = pd.DataFrame(cursor.fetchall(), columns = columns)   
df2.to_csv(t+'.csv', index=False)

In [ ]:
#read in table to check
# dbName = 'CHiMaD_metrics.db'
# tablename = 'loginMetrics'
#dbName = 'uchicago.db'
#dbName = 'dec2021_all.db'
#tablename = 'paragraphs'
#tablename = 'polymercompositeexample'
# dbName = 'available_dbs.db'
# tablename = 'dbs'
dbName = 'default.db'
tablename = 'polymercompositeexample'
#tablename = 'paragraphs'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

cursor.close()

In [ ]:
display(df)
df.replace({'groupname': {'testing': 'foo'}}, inplace = True)
display(df)

In [ ]:
#rename table
dbName = 'testing1.db'
tablename = 'polymercompositeexample'
newname = 'foo'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("ALTER TABLE `" + newname + "` RENAME TO `" + tablename + "`")
cursor.close()


In [ ]:
#rename value
paragraphname = 'Polymer Composite Example'
newname = 'foo'
db = 'testing1.db'
conn = sqlite3.connect(db)
cursor = conn.cursor()
cursor.execute('SELECT * FROM paragraphs')
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns) 
df.replace({'paragraphname': {paragraphname: newname}}, inplace = True)
df.to_sql('paragraphs', conn, if_exists='replace', index = False)
cursor.close()

In [ ]:
cursor.close()

In [ ]:
# command to copy files from Downloads and change names
#find ~/Downloads/ | grep testParagraph | awk '{print ("cp "$0" "$3)}' | sed s/" - "/"\\\ -\\\ "/ > foo